In [3]:
import sys
import tensorflow as tf
import json
import os
n_epoch = 3
import pandas as pd

In [7]:
import csv
from pandas.io.json import json_normalize
dirs = os.listdir("../../data/json/")
print len(dirs)
with open('../../data/tripadv.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerow(["Ratings","Content"])
    for filename in dirs:
        with open('../../data/json/'+filename) as fi:
            
            try:
                d = json.load(fi)
            except:
                d={}
                d["Reviews"] = []
        for x in d["Reviews"]:
            if float(x["Ratings"]["Overall"]) in [1.0,2.0,3.0,4.0,5.0]:
                writer.writerow([float(x["Ratings"]["Overall"].encode('utf-8')),x["Content"].encode('utf-8')])

1325


In [8]:
import pandas as pd
df = pd.read_csv('../../data/tripadv.csv')
df.head(10)

,Ratings,Content
0,5.0,"Usually stay near the airport, but this trip w..."
1,4.0,Stayed at this Hilton for 2 nights. It was lik...
2,4.0,"Stayed there one night, December 16, on the wa..."
3,5.0,I just stayed here last weekend and have alrea...
4,5.0,My mother who is 90 and I stayed one night on ...
5,4.0,"A fabulous stop. Clean, friendly, excellent lo..."
6,4.0,I stopped here just for the night on a road tr...
7,4.0,Great spot close to shops and dinning but perh...
8,4.0,My husband and I go to Baltimore often as he h...
9,5.0,Super clean. Great hotel to stay at. We actual...


In [3]:
X = df.Content
y = df.Ratings
X.head(10)
y.head(10)

0    5.0
1    4.0
2    4.0
3    5.0
4    5.0
5    4.0
6    4.0
7    4.0
8    4.0
9    5.0
Name: Ratings, dtype: float64

In [4]:
def get_positive_or_negative_classes_two_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 1)
def get_positive_or_negative_classes_three_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 2 if val > 3 else 1)
def get_positive_or_negative_classes_five_classes(y):
    return y.apply(lambda val: 0 if val=='' else val)

In [5]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
y_train = get_positive_or_negative_classes_three_classes(y_train)
y_test = get_positive_or_negative_classes_three_classes(y_test)

In [6]:
import string
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [7]:
vect.fit(X_train)
vocab = vect.vocabulary_
len(vocab)

132362

In [8]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [9]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)


In [13]:
X_train_word_ids.head()

87742     [61869, 117272, 96328, 91452, 103347, 53053, 6...
178022    [20262, 102137, 130008, 14055, 80615, 73925, 6...
214335    [57226, 4595, 118614, 126568, 117939, 127793, ...
115399    [128249, 112180, 117939, 61869, 43304, 118986,...
45982     [128249, 55082, 86553, 85218, 37384, 10077, 12...
Name: Content, dtype: object

In [17]:
X_train.head()

87742     Ancient, Budget-Minded Hotel The Ramada Plaza ...
178022    Beautiful rooms with american movie memorabili...
214335    Great 1st Time Visit This was my first visit t...
115399    We stayed this hotel due to a flight cancellat...
45982     We gave our olders daughter a trip to New York...
Name: Content, dtype: object

In [10]:
X_train_padded_seqs = pad_sequences(X_train_word_ids, maxlen=20, value=0)
X_test_padded_seqs  = pad_sequences(X_test_word_ids , maxlen=20, value=0)

In [11]:
print('X_train_padded_seqs.shape:', X_train_padded_seqs.shape)
print('X_test_padded_seqs.shape:', X_test_padded_seqs.shape)

('X_train_padded_seqs.shape:', (249050, 20))
('X_test_padded_seqs.shape:', (27673, 20))


In [37]:
pd.DataFrame(X_train_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,61869,117272,96328,91452,103347,53053,66643,83645,52456,13295,117939,66643,10077,126025,85205,61869,52456,120459,62984,58316
1,20262,102137,130008,14055,80615,73925,66825,0,0,0,0,0,0,0,0,0,0,0,0,0
2,57226,4595,118614,126568,117939,127793,81226,51245,126568,118986,82437,109690,53547,113974,118986,56335,43517,53326,95593,50598
3,128249,112180,117939,61869,43304,118986,10077,51742,117272,102048,66643,30439,14441,79581,117272,24973,105160,118986,20010,85205
4,128249,55082,86553,85218,37384,10077,120917,118986,82437,131691,52456,60174,56906,53718,60508,106351,59273,13783,127647,118986


In [38]:
pd.DataFrame(X_test_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,128249,100667,71238,128437,53718,10077,128437,64111,71200,125589,17194,117272,62984,124795,118986,51021,117272,21366,117551,33830
1,128249,58316,10077,73946,112153,17194,117272,61869,42991,14441,50444,17194,61231,117272,78921,81226,62784,14441,121817,29228
2,117272,78969,61869,27247,127793,66825,58316,117272,80386,110312,102048,84858,13295,117272,61977,128249,112180,126025,31845,20395
3,16717,10077,53368,126597,118986,62984,130728,97605,117939,61869,118986,15168,105151,10077,30439,61869,64111,117272,61869,66643
4,128249,99289,10077,232,20480,33021,40589,53718,117272,117272,99283,94081,127793,104880,14441,117272,87454,127793,126025,117272


In [12]:
unique_y_labels = list(y_train.value_counts().index)
unique_y_labels
unique_y_labels_test = list(y_test.value_counts().index)
unique_y_labels_test

[2, 0, 1]

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(unique_y_labels)

LabelEncoder()

In [14]:
y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), nb_classes=len(unique_y_labels))
y_test  = to_categorical(y_test.map(lambda x:  le.transform([x])[0]), nb_classes=len(unique_y_labels))

In [15]:
y_train

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [16]:
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

('y_train.shape:', (249050, 3))
('y_test.shape:', (27673, 3))


In [17]:
size_of_each_vector = X_train_padded_seqs.shape[1]
vocab_size = len(vocab)
no_of_unique_y_labels = len(unique_y_labels)

In [18]:
print('size_of_each_vector:', size_of_each_vector)
print('vocab_size:', vocab_size)
print('no_of_unique_y_labels:', no_of_unique_y_labels)

('size_of_each_vector:', 20)
('vocab_size:', 132362)
('no_of_unique_y_labels:', 3)


In [19]:
net = tflearn.input_data([None, size_of_each_vector]) # The first element is the "batch size" which we set to "None"
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128) # input_dim: vocabulary size
net = tflearn.lstm(net, 128, dropout=0.6) # Set the dropout to 0.6
net = tflearn.fully_connected(net, no_of_unique_y_labels, activation='softmax') # relu or softmax
net = tflearn.regression(net, 
                         optimizer='adam',  # adam or ada or adagrad # sgd
                         learning_rate=1e-4,
                         loss='categorical_crossentropy')

In [20]:
model = tflearn.DNN(net, tensorboard_verbose=0)

In [21]:
model.fit(X_train_padded_seqs, y_train, 
          validation_set=(X_test_padded_seqs, y_test), 
          n_epoch=n_epoch,
          show_metric=True, 
          batch_size=100)

Training Step: 7472  | total loss: 0.68779 | time: 787.549s
| Adam | epoch: 003 | loss: 0.68779 - acc: 0.7353 -- iter: 249000/249050
Training Step: 7473  | total loss: 0.68677 | time: 794.905s
| Adam | epoch: 003 | loss: 0.68677 - acc: 0.7338 | val_loss: 0.66764 - val_acc: 0.7367 -- iter: 249050/249050
--


In [22]:
import numpy as np
from sklearn import metrics

In [23]:
pred_classes = [np.argmax(i) for i in model.predict(X_test_padded_seqs)]
true_classes = [np.argmax(i) for i in y_test]
print('\nRNN Classifier\'s Accuracy: %0.5f\n' % metrics.accuracy_score(true_classes, pred_classes))




RNN Classifier's Accuracy: 0.73671



In [24]:
print ('Confusion Matrix:', metrics.confusion_matrix(le.inverse_transform(true_classes), le.inverse_transform(pred_classes)))
print ('Classification Report')
print metrics.classification_report(le.inverse_transform(true_classes), le.inverse_transform(pred_classes))

('Confusion Matrix:', array([[ 2079,     0,  2462],
       [  840,     0,  3203],
       [  781,     0, 18308]]))
Classification Report
             precision    recall  f1-score   support

          0       0.56      0.46      0.50      4541
          1       0.00      0.00      0.00      4043
          2       0.76      0.96      0.85     19089

avg / total       0.62      0.74      0.67     27673



/Users/savinaynarendra/anaconda2/envs/cs670/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
ids_of_titles = range(0,100) # range(X_test.shape[0]) 

for i in ids_of_titles:
    pred_class = np.argmax(model.predict([X_test_padded_seqs[i]]))
    true_class = np.argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class:', le.inverse_transform(pred_class))
    print('true_class:', le.inverse_transform(true_class))
    print('')

We returned last week from a week in Las Vegas at the Orleans. I used travelaxe to find the best rate, then contacted the hotel directly, and they gladly matched the rate. They even took off the commission that the booking company would have received that is built into their price. Result: room was $5 per night cheaper than the lowest competitor's price. The Orleans was the best hotel in 7 trips that I have stayed in. Everyone from check in to check out was friendly and helpful. The room was a standard non-smoking room on the 10th floor. It also looked as though it had just been remodeled. EVERYTHING looked new. Very, very nice. It seemed roomier than when I stayed at the Golden Nugget. It is also the first trip that I've taken and come back with more money than I left with. Some was from blackjack, the rest from nickle video poker. Second day there, I hit a progressive royal flush for $432.10 (not bad for nickles) and must have had at least 30 sets of quads--all on the same machine al